In [2]:
import numpy as np
import pandas as pd
import os

In [2]:
path_hitter2023 = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/gamedata/hitter_2023.csv'
path_pitcher2023 = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/gamedata/pitcher_2023.csv'
path_hitterRecords = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/hitterRecords.csv'
path_pitcherRecords = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/pitcherRecords.csv'
path_teamRank = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/KBO_Team_rank_stat_v.01.csv'
path_pitcherDetailed = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/pitcher_detailed_stats.csv'
path_hitterDetailed = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/hitter_detailed_stats.csv'
path_playerInfo = '/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/playerInfo_v.01.xlsx'

## 0. 로스터 변경사항 

- In

    안치홍 (내야수, 롯데) / 이상규 (투수, LG) / 배민서 (투수, NC) / 김강민 (외야수, SSG) / 이재원 (포수, SSG) / 김도빈 (투수, 수원)

- Out

    오선진 (내야수)

- 계약 종료

    이재민, 이석제, 장웅정, 천보웅, 신현수, 이준기, 김재영, 류원석, 류희운, 박윤철, 송윤준 (이상 투수) / 고영재, 원혁재, 이정재, 노수광, 유상빈, 장운호


신인드래프트에선 1-2픽 투수, 3-4픽 내야수를 영입했다.

## 1. 2023년 경기별 기록 전처리

In [37]:
hitter2023 = pd.read_csv(path_hitter2023, index_col=0)
pitcher2023 = pd.read_csv(path_pitcher2023, index_col=0)

In [38]:
def parse_inning(inning_str):
    # 이닝 문자열을 공백으로 분할하여 정수 부분과 분수 부분을 분리합니다.
    parts = inning_str.split()

    # 정수 부분만 있는 경우 (예: "5")
    if len(parts) == 1:
        if parts[0]  == '1/3':
            return 0.3
        elif parts[0] == '2/3':
            return 0.7
        else:
            return float(parts[0])
    
    # 정수와 분수가 모두 있는 경우 (예: "2 ⅓")
    elif len(parts) == 2:
        whole_part = float(parts[0])  # 정수 부분
        fraction_part = parts[1]  # 분수 부분

        # 분수를 부동소수점 값으로 변환합니다.
        if fraction_part == '⅓' or fraction_part  == '1/3':
            fraction_value = 0.3
        elif fraction_part == '⅔' or fraction_part == '2/3':
            fraction_value = 0.7
        else:
            # 지원되지 않는 분수 형태인 경우 오류를 발생시킵니다.
            raise ValueError(f"Unsupported fraction part: {fraction_part}")

        return whole_part + fraction_value

    else:
        # 잘못된 형식의 입력인 경우 오류를 발생시킵니다.
        raise ValueError(f"Invalid inning format: {inning_str}")



In [39]:
pitcher2023['이닝'] = pitcher2023['이닝'].apply(parse_inning)

In [40]:
pitcherint = ['경기', '승리', '패전', '세이브']
for col in pitcherint: 
    pitcher2023[col] = pitcher2023[col].astype(int)
pitcher2023.dtypes    

투수명         object
이닝         float64
피안타          int64
실점           int64
자책           int64
4사구          int64
삼진           int64
피홈런          int64
타자           int64
타수           int64
투구수          int64
경기           int64
승리           int64
패전           int64
세이브          int64
평균자책       float64
연도           int64
월            int64
일            int64
소속팀         object
상대팀         object
경기장         object
matchid     object
dtype: object

In [41]:
pitcher2023

,투수명,이닝,피안타,실점,자책,4사구,삼진,피홈런,타자,타수,...,패전,세이브,평균자책,연도,월,일,소속팀,상대팀,경기장,matchid
0,이정용,3.0,3,0,0,1,1,0,14,13,...,0,3,4.85,2023,7,2,LG,KIA,홈,20230702HTLG02023
1,정우영,1.0,0,0,0,0,0,0,3,3,...,4,0,4.24,2023,7,2,LG,KIA,홈,20230702HTLG02023
2,이우찬,2.0,2,0,0,0,0,0,6,6,...,1,0,2.89,2023,7,2,LG,KIA,홈,20230702HTLG02023
3,최동환,0.3,2,1,1,0,1,0,3,3,...,0,0,2.75,2023,7,2,LG,KIA,홈,20230702HTLG02023
4,함덕주,1.7,2,0,0,0,2,0,7,7,...,0,3,1.15,2023,7,2,LG,KIA,홈,20230702HTLG02023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6829,백승건,1.0,1,0,0,0,0,0,4,4,...,1,0,3.15,2023,5,23,SSG,LG,홈,20230523LGSK02023
6830,임찬규,6.0,3,1,1,1,3,1,22,21,...,0,0,2.33,2023,5,23,SSG,LG,원정,20230523LGSK02023
6831,유영찬,1.0,0,0,0,1,0,0,4,3,...,1,0,3.74,2023,5,23,SSG,LG,원정,20230523LGSK02023
6832,진해수,1.0,1,0,0,0,2,0,4,4,...,0,0,3.18,2023,5,23,SSG,LG,원정,20230523LGSK02023


In [42]:
# 팀 약어 목록
team_abbreviations = ['LT', 'SK', 'HH', 'WO', 'SS', 'NC', 'LG', 'HT', 'OB', 'KT']

# 각 팀별로 첫 번째 선발투수 데이터를 저장할 빈 데이터프레임 생성
first_pitchers_by_team = {team_abbr: pd.DataFrame() for team_abbr in team_abbreviations}

# 각 팀에 대한 데이터프레임을 순회하며 첫 번째 선발투수 데이터 추출
for team_abbr in team_abbreviations:
    for _, row in pitcher2023.iterrows():
        matchid = row['matchid']
        team_part = matchid[10:12]
        away_team = team_part[8:10]

        # 해당 팀이 원정팀인 경우
        if away_team == team_abbr: 
            team_pitchers = pitcher2023[(pitcher2023['matchid'] == matchid) & (pitcher2023['경기장'] == '원정')]
        # 해당 팀이 홈팀인 경우
        else:
            team_pitchers = pitcher2023[(pitcher2023['matchid'] == matchid) & (pitcher2023['경기장'] == '홈')]

        # 첫 번째 선발투수 데이터가 있는 경우 해당 팀의 데이터프레임에 추가
        if not team_pitchers.empty:
            first_pitchers_by_team[team_abbr] = pd.concat([first_pitchers_by_team[team_abbr], team_pitchers.iloc[[0]]], ignore_index=True)

# 결과 확인 (예시: 'LT' 팀의 첫 번째 선발투수 데이터의 첫 행 출력)
print(first_pitchers_by_team['LT'])

      투수명   이닝  피안타  실점  자책  4사구  삼진  피홈런  타자  타수  ...  패전  세이브  평균자책    연도  \
0     이정용  3.0    3   0   0    1   1    0  14  13  ...   0    3  4.85  2023   
1     이정용  3.0    3   0   0    1   1    0  14  13  ...   0    3  4.85  2023   
2     이정용  3.0    3   0   0    1   1    0  14  13  ...   0    3  4.85  2023   
3     이정용  3.0    3   0   0    1   1    0  14  13  ...   0    3  4.85  2023   
4     이정용  3.0    3   0   0    1   1    0  14  13  ...   0    3  4.85  2023   
...   ...  ...  ...  ..  ..  ...  ..  ...  ..  ..  ...  ..  ...   ...   ...   
6815  오원석  5.0    6   6   6    5   2    1  24  19  ...   2    0  3.73  2023   
6816  오원석  5.0    6   6   6    5   2    1  24  19  ...   2    0  3.73  2023   
6817  오원석  5.0    6   6   6    5   2    1  24  19  ...   2    0  3.73  2023   
6818  오원석  5.0    6   6   6    5   2    1  24  19  ...   2    0  3.73  2023   
6819  오원석  5.0    6   6   6    5   2    1  24  19  ...   2    0  3.73  2023   

      월   일  소속팀  상대팀  경기장            matchid  
0  

In [64]:
first_pitchers = pd.DataFrame()
for team in first_pitchers_by_team.keys():
    df = pd.DataFrame(first_pitchers_by_team[team])
    first_pitchers = pd.concat([first_pitchers, df], ignore_index=False)

In [65]:
first_pitchers = first_pitchers[['투수명', '소속팀']].to_numpy()

In [66]:
# 배열의 각 행을 튜플로 변환하고, 집합을 사용하여 고유한 행을 찾습니다.
unique_tuples = set(tuple(row) for row in first_pitchers)

# 고유한 튜플들을 다시 NumPy 배열로 변환합니다.
first_pitchers = np.array(list(unique_tuples))

print(first_pitchers)

[['한승주' '한화']
 ['파노니' 'KIA']
 ['이인복' '롯데']
 ['양창섭' '삼성']
 ['켈리' 'LG']
 ['송명기' 'NC']
 ['김민우' '한화']
 ['배제성' 'KT']
 ['맥키니' '키움']
 ['딜런' '두산']
 ['남지민' '한화']
 ['페냐' '한화']
 ['황동하' 'KIA']
 ['앤더슨' 'KIA']
 ['양현종' 'KIA']
 ['백승건' 'SSG']
 ['메디나' 'KIA']
 ['최원태' 'LG']
 ['이재학' 'NC']
 ['강효종' 'LG']
 ['조이현' 'KT']
 ['구창모' 'NC']
 ['산체스' '한화']
 ['와이드너' '삼성']
 ['임찬규' 'LG']
 ['안우진' '키움']
 ['심재민' '롯데']
 ['이민호' 'LG']
 ['알칸타라' '두산']
 ['오원석' 'SSG']
 ['후라도' '키움']
 ['원태인' '삼성']
 ['한승혁' '한화']
 ['이용준' 'NC']
 ['슐서' 'KT']
 ['손주영' 'LG']
 ['정성종' '롯데']
 ['최승용' '두산']
 ['최채흥' '삼성']
 ['오승환' '삼성']
 ['최원준' '두산']
 ['윤영철' 'KIA']
 ['주권' 'KT']
 ['요키시' '키움']
 ['곽빈' '두산']
 ['김광현' 'SSG']
 ['김선기' '키움']
 ['오상원' '키움']
 ['스트레일리' '롯데']
 ['김건국' 'KIA']
 ['김동주' '두산']
 ['이정용' 'LG']
 ['최원태' '키움']
 ['장필준' '삼성']
 ['이의리' 'KIA']
 ['김민규' '두산']
 ['반즈' '롯데']
 ['장민재' '한화']
 ['박세웅' '롯데']
 ['이상영' 'LG']
 ['이태양' '한화']
 ['김민' 'KT']
 ['최성영' 'NC']
 ['백정현' '삼성']
 ['브랜든' '두산']
 ['윌커슨' '롯데']
 ['장원준' '두산']
 ['황동재' '삼성']
 ['박신지' '두산']
 ['김기중' '한화']
 ['산체스' 'KIA'

In [67]:
first_pitchers = pd.DataFrame(first_pitchers, columns= ['이름', '팀명'])

In [72]:
first_pitchers['포지션'] = '선발'

## 2. 2023년 전체 기록 데이터 전처리

In [10]:
#hitterTotal = pd.read_csv('./dataset/hitterRecords.csv', index_col=0)
pitcherTotal = pd.read_csv('pitcher.csv', index_col=0)

In [132]:
#pitcherTotal = pitcherTotal.loc[pitcherTotal['연도'] == 2023]
hitterTotal = hitterTotal.loc[hitterTotal['연도'] == 2023]

In [7]:
pitcherTotal['이닝'] = pitcherTotal['이닝'].apply(parse_inning)
pitcherTotal = pitcherTotal.loc[pitcherTotal['승률'] != '-']
for col in ['평균자책점', '승률']:
    pitcherTotal[col] = pitcherTotal[col].astype(float)

In [133]:
hitterTotal = hitterTotal.loc[hitterTotal['타율'] != '-']
for col in ['타율', '장타율', '출루율']:
    hitterTotal[col] = hitterTotal[col].astype(float)

In [136]:
hitterTotal

,이름,id,연도,팀명,타율,경기,타석,타수,득점,안타,...,도루,도루실패,볼넷,사구,삼진,병살타,장타율,출루율,실책,생년월일
0,문상준,50007,2023,KT,0.200,2,5,5,1,1,...,0,0,0,0,4,0,0.200,0.200,1,2001-03-14
1,강현우,50066,2023,KT,0.194,53,111,103,11,20,...,0,1,6,1,17,3,0.282,0.245,2,2001-04-13
2,손호영,50150,2023,LG,0.205,27,45,44,8,9,...,2,1,1,0,12,5,0.273,0.222,5,1994-08-23
3,안권수,50202,2023,롯데,0.269,95,309,268,42,72,...,16,7,30,1,40,4,0.321,0.341,3,1993-04-19
4,이주형,50167,2023,키움,0.326,69,243,215,32,70,...,3,1,19,5,53,4,0.507,0.390,0,2001-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,국해성,79290,2023,롯데,0.250,6,10,8,0,2,...,0,0,2,0,3,0,0.250,0.400,0,1989-10-08
288,박동원,79365,2023,LG,0.249,130,481,409,54,102,...,0,1,49,7,90,7,0.443,0.334,8,1990-04-07
289,김상수,79402,2023,KT,0.271,129,512,443,58,120,...,5,6,55,3,68,9,0.339,0.353,6,1990-03-23
290,이지영,79456,2023,키움,0.249,81,237,217,23,54,...,1,1,12,1,39,2,0.295,0.291,4,1986-02-27


In [135]:
#pitcherTotal = pitcherTotal.reset_index()
hitterTotal = hitterTotal.reset_index()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



playername = [[name, birth] for name, birth in hitterTotal[['이름', '생년월일']].to_numpy()]
playerInfo = []
for i, player in enumerate(playername):
    url = 'http://www.statiz.co.kr/player.php?birth=' + str(player[1]) + '&name=' + str(player[0])
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    postext = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-4.col-md-4.col-lg-4 > div')
    pos = postext[0].text.split('\n')[-2]
    if pos[-1] == "택":
        url = 'http://www.statiz.co.kr/player.php?name=' + str(player[0])
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        postext = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-4.col-md-4.col-lg-4 > div')
        pos = postext[0].text.split('\n')[-2]
    pos = pos.split(', ')[1]
    info = player + [pos]
    playerInfo.append(info)
    if i % 100 == 0:
        print(f"epoch {i} is finished")

playerInfo

In [ ]:
hitterInfo = pd.DataFrame(playerInfo, columns = ['이름', '생년월일', '포지션'])
hitterInfo.loc[hitterInfo['포지션'] == '?', '포지션'] =  '3루수'
hitterInfo.loc[hitterInfo['포지션'] == '?']
hitterInfo.to_csv('hitterInfo.csv', index=False)

In [138]:
hitterInfo = pd.read_csv('./dataset/hitterInfo.csv', index_col=False)

In [139]:
hitterTotal = pd.merge(hitterTotal, hitterInfo, on= ['이름', '생년월일'], how = 'left')
hitterTotal['포지션'].unique()

array(['2루수', '포수', '유격수', '좌익수', '중견수', '1루수', '우익수', '지명타자', '투수',
       '3루수'], dtype=object)

In [140]:
hitterDetailed = pd.read_csv(path_hitterDetailed, index_col= False)
hitterDetailed = hitterDetailed.loc[hitterDetailed['연도'] == 2023]
hitterDetailed.loc[hitterDetailed['name'] == '이주형']

,name,id,birthday,연도,팀,나이,P,G,타석,타수,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
53,이주형,50167,2001-04-02,2023,"LG,키움",22,CF,69,243,215,...,2,2,0.326,0.390,0.507,0.897,0.405,151.8,2.60,1.39
54,이주형,50167,2001-04-02,2023,LG,22,CF,18,16,15,...,0,0,0.267,0.313,0.400,0.713,0.333,103.2,0.07,-0.05
55,이주형,50167,2001-04-02,2023,키움,22,CF,51,227,200,...,2,2,0.330,0.396,0.515,0.911,0.410,155.4,2.37,1.45


In [141]:
hitterDetailed = hitterDetailed.rename(columns= {'name' : '이름', 'birthday' : '생년월일', '팀' : '팀명', '희비' : '희생플라이' })
#hitterDetailed.to_csv('hitterDetailed.csv')

In [142]:
hitterTotal.columns

Index(['이름', 'id', '연도', '팀명', '타율', '경기', '타석', '타수', '득점', '안타', '2루타',
       '3루타', '홈런', '루타', '타점', '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '장타율',
       '출루율', '실책', '생년월일', '포지션'],
      dtype='object')

In [143]:
hitterTotal = pd.merge(hitterTotal, hitterDetailed[['이름', '생년월일', '팀명', '희생플라이', 'WAR*']], on=['이름', '생년월일', '팀명'], how='left')
hitterTotal

,이름,id,연도,팀명,타율,경기,타석,타수,득점,안타,...,사구,삼진,병살타,장타율,출루율,실책,생년월일,포지션,희생플라이,WAR*
0,문상준,50007,2023,KT,0.200,2,5,5,1,1,...,0,4,0,0.200,0.200,1,2001-03-14,2루수,0.0,-0.05
1,강현우,50066,2023,KT,0.194,53,111,103,11,20,...,1,17,3,0.282,0.245,2,2001-04-13,포수,0.0,-0.12
2,손호영,50150,2023,LG,0.205,27,45,44,8,9,...,0,12,5,0.273,0.222,5,1994-08-23,유격수,0.0,-0.13
3,안권수,50202,2023,롯데,0.269,95,309,268,42,72,...,1,40,4,0.321,0.341,3,1993-04-19,좌익수,3.0,0.74
4,이주형,50167,2023,키움,0.326,69,243,215,32,70,...,5,53,4,0.507,0.390,0,2001-04-02,중견수,2.0,2.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,국해성,79290,2023,롯데,0.250,6,10,8,0,2,...,0,3,0,0.250,0.400,0,1989-10-08,지명타자,0.0,0.03
288,박동원,79365,2023,LG,0.249,130,481,409,54,102,...,7,90,7,0.443,0.334,8,1990-04-07,포수,8.0,3.41
289,김상수,79402,2023,KT,0.271,129,512,443,58,120,...,3,68,9,0.339,0.353,6,1990-03-23,유격수,3.0,2.29
290,이지영,79456,2023,키움,0.249,81,237,217,23,54,...,1,39,2,0.295,0.291,4,1986-02-27,포수,0.0,0.40


In [144]:
hitterTotal.loc[hitterTotal['희생플라이'].isna()]

,이름,id,연도,팀명,타율,경기,타석,타수,득점,안타,...,사구,삼진,병살타,장타율,출루율,실책,생년월일,포지션,희생플라이,WAR*
42,조민성,52405,2023,삼성,0.207,15,29,29,1,6,...,0,6,1,0.276,0.207,0,2003-10-22,1루수,NaN,NaN


In [145]:
hitterTotal.loc[hitterTotal['이름'] == '조민성', 'WAR*'] = -0.37
hitterTotal.loc[hitterTotal['이름'] == '조민성', '희생플라이'] = 0

In [146]:
hitterTotal.to_csv('./dataset/hitterTotal.csv', index=False)

In [11]:
pitcherTotal = pitcherTotal.reset_index()
pitcherTotal

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,피안타,피홈런,볼넷,사구,삼진,실점,자책점,생년월일,포지션,WAR
0,유영찬,50106,2023,LG,3.44,67,0,0,6,3,...,55,4,40,7,55,27,26,1997-03-07,중계,1.55
1,이민호,50126,2023,LG,5.03,5,0,0,0,2,...,22,1,10,0,7,13,11,2001-08-30,선발,-0.20
2,김윤식,50157,2023,LG,4.22,17,0,0,6,4,...,86,4,25,3,42,35,35,2000-04-03,선발,1.00
3,이종민,50354,2023,키움,7.24,11,0,0,0,1,...,22,0,6,1,10,13,11,2001-06-04,중계,-0.36
4,김동혁,50360,2023,키움,7.32,35,0,0,1,7,...,51,1,18,6,24,34,32,2001-12-27,선발,-0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,장필준,77927,2023,삼성,7.91,17,0,0,1,1,...,28,2,12,2,11,17,17,1988-04-08,선발,-0.30
197,정찬헌,78148,2023,키움,4.75,14,1,0,2,8,...,71,4,13,2,35,40,38,1990-01-26,선발,0.67
198,임창민,78352,2023,키움,2.51,51,0,0,2,2,...,51,3,17,2,40,16,13,1985-08-25,중계,1.89
199,하준호,78517,2023,KT,4.15,12,0,0,0,1,...,14,0,10,5,6,9,6,1989-04-29,중계,0.04


In [8]:
pitcherDetailed = pd.read_csv('/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/김영호/dataset/pitcherDetailed.csv', index_col= False)
pitcherDetailed = pitcherDetailed.loc[pitcherDetailed['연도'] == 2023]
pitcherDetailed

,이름,id,생년월일,연도,팀명,나이,출장,완투,완봉,선발,...,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA,포지션
0,소형준,50030,2001-09-16,2023,KT,22,3,0,0,3,...,0,2,11.45,5.53,2.27,36.9,75.0,-0.53,-0.52,선발
1,유영찬,50106,1997-03-07,2023,LG,26,67,0,0,0,...,0,8,3.44,4.58,1.40,118.6,89.5,1.55,-0.12,중계
2,이민호,50126,2001-08-30,2023,LG,22,5,0,0,5,...,0,0,5.03,4.92,1.63,81.1,83.4,-0.20,-0.43,선발
3,김윤식,50157,2000-04-03,2023,LG,23,17,0,0,16,...,0,4,4.22,4.14,1.49,96.7,98.9,1.00,0.40,선발
4,제환유,50211,2000-09-30,2023,두산,23,1,0,0,0,...,0,1,13.50,7.44,3.50,30.2,55.7,-0.11,-0.01,중계
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,정찬헌,78148,1990-01-26,2023,키움,33,14,1,0,14,...,0,0,4.75,3.78,1.17,86.5,108.1,0.67,-0.15,선발
292,임창민,78352,1985-08-25,2023,키움,38,51,0,0,0,...,0,0,2.51,3.59,1.46,163.8,113.1,1.89,0.55,중계
293,하준호,78517,1989-04-29,2023,KT,34,12,0,0,0,...,0,4,4.85,5.52,1.85,87.1,75.5,0.04,-0.15,중계
294,최동환,79140,1989-09-19,2023,LG,34,45,0,0,0,...,0,0,3.19,4.08,1.39,128.0,101.1,0.74,0.18,중계


In [121]:
pitcherDetailed = pitcherDetailed.rename(columns= {'name' : '이름', 'birthday' : '생년월일', '팀' : '팀명'})
pitcherDetailed

,이름,id,생년월일,연도,팀명,나이,출장,완투,완봉,선발,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
16,소형준,50030,2001-09-16,2023,KT,22,3,0,0,3,...,4,0,2,11.45,5.53,2.27,36.9,75.0,-0.53,-0.52
25,유영찬,50106,1997-03-07,2023,LG,26,67,0,0,0,...,55,0,8,3.44,4.58,1.40,118.6,89.5,1.55,-0.12
30,이민호,50126,2001-08-30,2023,LG,22,5,0,0,5,...,7,0,0,5.03,4.92,1.63,81.1,83.4,-0.20,-0.43
34,김윤식,50157,2000-04-03,2023,LG,23,17,0,0,16,...,42,0,4,4.22,4.14,1.49,96.7,98.9,1.00,0.40
40,제환유,50211,2000-09-30,2023,두산,23,1,0,0,0,...,2,0,1,13.50,7.44,3.50,30.2,55.7,-0.11,-0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,정찬헌,78148,1990-01-26,2023,키움,33,14,1,0,14,...,35,0,0,4.75,3.78,1.17,86.5,108.1,0.67,-0.15
4052,임창민,78352,1985-08-25,2023,키움,38,51,0,0,0,...,40,0,0,2.51,3.59,1.46,163.8,113.1,1.89,0.55
4074,하준호,78517,1989-04-29,2023,KT,34,12,0,0,0,...,6,0,4,4.85,5.52,1.85,87.1,75.5,0.04,-0.15
4163,최동환,79140,1989-09-19,2023,LG,34,45,0,0,0,...,20,0,0,3.19,4.08,1.39,128.0,101.1,0.74,0.18


In [82]:
pitcherDetailed = pd.merge(pitcherDetailed, first_pitchers, on = ['이름', '팀명'], how='left')
pitcherDetailed

,이름,id,생년월일,연도,팀명,나이,출장,완투,완봉,선발,...,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA,포지션
0,소형준,50030,2001-09-16,2023,KT,22,3,0,0,3,...,0,2,11.45,5.53,2.27,36.9,75.0,-0.53,-0.52,선발
1,유영찬,50106,1997-03-07,2023,LG,26,67,0,0,0,...,0,8,3.44,4.58,1.40,118.6,89.5,1.55,-0.12,NaN
2,이민호,50126,2001-08-30,2023,LG,22,5,0,0,5,...,0,0,5.03,4.92,1.63,81.1,83.4,-0.20,-0.43,선발
3,김윤식,50157,2000-04-03,2023,LG,23,17,0,0,16,...,0,4,4.22,4.14,1.49,96.7,98.9,1.00,0.40,선발
4,제환유,50211,2000-09-30,2023,두산,23,1,0,0,0,...,0,1,13.50,7.44,3.50,30.2,55.7,-0.11,-0.01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,정찬헌,78148,1990-01-26,2023,키움,33,14,1,0,14,...,0,0,4.75,3.78,1.17,86.5,108.1,0.67,-0.15,선발
292,임창민,78352,1985-08-25,2023,키움,38,51,0,0,0,...,0,0,2.51,3.59,1.46,163.8,113.1,1.89,0.55,NaN
293,하준호,78517,1989-04-29,2023,KT,34,12,0,0,0,...,0,4,4.85,5.52,1.85,87.1,75.5,0.04,-0.15,NaN
294,최동환,79140,1989-09-19,2023,LG,34,45,0,0,0,...,0,0,3.19,4.08,1.39,128.0,101.1,0.74,0.18,NaN


In [83]:
pitcherDetailed

,이름,id,생년월일,연도,팀명,나이,출장,완투,완봉,선발,...,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA,포지션
0,소형준,50030,2001-09-16,2023,KT,22,3,0,0,3,...,0,2,11.45,5.53,2.27,36.9,75.0,-0.53,-0.52,선발
1,유영찬,50106,1997-03-07,2023,LG,26,67,0,0,0,...,0,8,3.44,4.58,1.40,118.6,89.5,1.55,-0.12,NaN
2,이민호,50126,2001-08-30,2023,LG,22,5,0,0,5,...,0,0,5.03,4.92,1.63,81.1,83.4,-0.20,-0.43,선발
3,김윤식,50157,2000-04-03,2023,LG,23,17,0,0,16,...,0,4,4.22,4.14,1.49,96.7,98.9,1.00,0.40,선발
4,제환유,50211,2000-09-30,2023,두산,23,1,0,0,0,...,0,1,13.50,7.44,3.50,30.2,55.7,-0.11,-0.01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,정찬헌,78148,1990-01-26,2023,키움,33,14,1,0,14,...,0,0,4.75,3.78,1.17,86.5,108.1,0.67,-0.15,선발
292,임창민,78352,1985-08-25,2023,키움,38,51,0,0,0,...,0,0,2.51,3.59,1.46,163.8,113.1,1.89,0.55,NaN
293,하준호,78517,1989-04-29,2023,KT,34,12,0,0,0,...,0,4,4.85,5.52,1.85,87.1,75.5,0.04,-0.15,NaN
294,최동환,79140,1989-09-19,2023,LG,34,45,0,0,0,...,0,0,3.19,4.08,1.39,128.0,101.1,0.74,0.18,NaN


In [84]:
pitcherDetailed.loc[pitcherDetailed['포지션'].isna(), '포지션'] = '중계'
pitcherDetailed

,이름,id,생년월일,연도,팀명,나이,출장,완투,완봉,선발,...,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA,포지션
0,소형준,50030,2001-09-16,2023,KT,22,3,0,0,3,...,0,2,11.45,5.53,2.27,36.9,75.0,-0.53,-0.52,선발
1,유영찬,50106,1997-03-07,2023,LG,26,67,0,0,0,...,0,8,3.44,4.58,1.40,118.6,89.5,1.55,-0.12,중계
2,이민호,50126,2001-08-30,2023,LG,22,5,0,0,5,...,0,0,5.03,4.92,1.63,81.1,83.4,-0.20,-0.43,선발
3,김윤식,50157,2000-04-03,2023,LG,23,17,0,0,16,...,0,4,4.22,4.14,1.49,96.7,98.9,1.00,0.40,선발
4,제환유,50211,2000-09-30,2023,두산,23,1,0,0,0,...,0,1,13.50,7.44,3.50,30.2,55.7,-0.11,-0.01,중계
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,정찬헌,78148,1990-01-26,2023,키움,33,14,1,0,14,...,0,0,4.75,3.78,1.17,86.5,108.1,0.67,-0.15,선발
292,임창민,78352,1985-08-25,2023,키움,38,51,0,0,0,...,0,0,2.51,3.59,1.46,163.8,113.1,1.89,0.55,중계
293,하준호,78517,1989-04-29,2023,KT,34,12,0,0,0,...,0,4,4.85,5.52,1.85,87.1,75.5,0.04,-0.15,중계
294,최동환,79140,1989-09-19,2023,LG,34,45,0,0,0,...,0,0,3.19,4.08,1.39,128.0,101.1,0.74,0.18,중계


In [90]:
pitcherTotal.columns, pitcherDetailed.columns

(Index(['이름', 'id', '연도', '팀명', '평균자책점', '경기', '완투', '완봉', '승리', '패배', '세이브',
        '홀드', '승률', '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점',
        '생년월일'],
       dtype='object'),
 Index(['이름', 'id', '생년월일', '연도', '팀명', '나이', '출장', '완투', '완봉', '선발', '승', '패',
        '세', '홀드', '이닝', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4',
        '사구', '삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR',
        'WPA', '포지션'],
       dtype='object'))

In [12]:
pitcherTotal = pd.merge(pitcherTotal, pitcherDetailed[['이름', '생년월일', '팀명', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WPA']], on=['이름', '생년월일', '팀명'], how='left')
pitcherTotal

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,생년월일,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA
0,유영찬,50106,2023,LG,3.44,67,0,0,6,3,...,1997-03-07,중계,1.55,8.0,3.44,4.58,1.40,118.6,89.5,-0.12
1,이민호,50126,2023,LG,5.03,5,0,0,0,2,...,2001-08-30,선발,-0.20,0.0,5.03,4.92,1.63,81.1,83.4,-0.43
2,김윤식,50157,2023,LG,4.22,17,0,0,6,4,...,2000-04-03,선발,1.00,4.0,4.22,4.14,1.49,96.7,98.9,0.40
3,이종민,50354,2023,키움,7.24,11,0,0,0,1,...,2001-06-04,중계,-0.36,0.0,7.24,3.52,2.05,56.7,117.1,-0.71
4,김동혁,50360,2023,키움,7.32,35,0,0,1,7,...,2001-12-27,선발,-0.81,0.0,7.55,4.38,1.75,54.4,93.7,-1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,장필준,77927,2023,삼성,7.91,17,0,0,1,1,...,1988-04-08,선발,-0.30,0.0,7.91,5.82,2.07,53.5,71.8,-0.34
197,정찬헌,78148,2023,키움,4.75,14,1,0,2,8,...,1990-01-26,선발,0.67,0.0,4.75,3.78,1.17,86.5,108.1,-0.15
198,임창민,78352,2023,키움,2.51,51,0,0,2,2,...,1985-08-25,중계,1.89,0.0,2.51,3.59,1.46,163.8,113.1,0.55
199,하준호,78517,2023,KT,4.15,12,0,0,0,1,...,1989-04-29,중계,0.04,4.0,4.85,5.52,1.85,87.1,75.5,-0.15


In [13]:
pitcherTotal.loc[pitcherTotal['WAR'].isna()]

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,생년월일,포지션,WAR,폭투,ERA,FIP,WHIP,ERA+,FIP+,WPA
44,페냐,52720,2023,한화,3.6,32,0,0,11,11,...,1990-01-25,선발,3.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'WAR'] = 3.53

In [18]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', '폭투'] = 16

In [19]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'ERA'] = 3.60

In [20]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'FIP'] = 4.10

In [21]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'WHIP'] = 1.17

In [22]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'ERA+'] = 115.2

In [23]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'FIP+'] = 101.3

In [24]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', 'WPA'] = -0.27

In [25]:
pitcherTotal.loc[pitcherTotal['이름'] == '페냐', '포지션'] = '선발'

In [28]:
pitcherTotal.to_csv('./dataset/pitcherTotal.csv', index=False)
#pitcherDetailed.to_csv('pitcherDetailed.csv', index=False)

타자의 경우 실질적으로 포지션을 알 수 없는 경우 = 주전이 무조건 아니다!

제외하고, 주전만 보겠다!

투수의 경우는?

전개 방향

1. 득점 / 실점 (Q0에서 어떻게 살펴봤는지 보자)
2. 타자는 포지션별로! (타율 / 타점)
3. 수비는 포지션별로! (실책)
4. 전체적인 WAR 지표로 
5. 투수는 선발 vs 중계 vs 마무리
6. 방어율/승/홀드/세이브 등등등..
7. WAR

그러면 지금 뭐가 부족한지 알 수 있을 것이다!

새로 들어온 선수는 이걸 채울수 있느냐?
아니라면, 새로 들어온 선수를 포함한 기록 측정을 다시 하였을 때 각 기록이 상승이 되느냐? 
